In [ ]:
import pandas as pd
import ixmp4
import nomenclature
import pyam

In [ ]:
iamc_args = dict(
    model="State of CDR (2024) - Demonstration and Upscaling",
    scenario="Observed Data",
)

In [ ]:
dsd = nomenclature.DataStructureDefinition("../definitions/")

In [ ]:
data_start_up = pd.read_excel("source/3_Demonstration/chapter-3-start-up-template.xlsx")

In [ ]:
data_start_up["Unit"].fillna("", inplace=True)
data_start_up.drop(columns="Description", inplace=True)

In [ ]:
data_start_up_1 = data_start_up[
    [("Start-Up Investment|Deals" == v or "Start-Up Investment|Funding" == v or "Share" in v) for v in data_start_up.Variable]
]

In [ ]:
data_start_up_2 = data_start_up[
    [("Start-Up Investment|Deals" != v and "Start-Up Investment|Funding" != v and "Share" not in v) for v in data_start_up.Variable]
].fillna(0)

In [ ]:
df = pyam.concat(
    [data_start_up_1, data_start_up_2],
    **iamc_args,
)

In [ ]:
df.rename(
    variable={
        "Start-Up Investment|Deals": "Start-Up|All Net-Zero Initiatives|Deals",
        "Start-Up Investment|Funding": "Start-Up|All Net-Zero Initiatives|Funding",
    },
    inplace=True,
)

In [ ]:
df.rename(
    variable=dict(
        {
          (
              i, 
              (
                  i.replace("|Removal", "")
                  .replace("Start-Up Investment", "Start-Up|Removal")
                  .replace("BECCS", "Bioenergy with CCS")
                  .replace("DAC(CS)", "Direct Air Capture with CCS")
                  .replace("DOC(CS)", "Direct Ocean Capture")
                  .replace("Soil Carbon", "Soil Carbon Sequestration")
              )
          )
            for i in df.variable
        },
    ),
    inplace=True,
)

In [ ]:
startup_number = pd.read_csv("source/3_Demonstration/fig2a_v2.csv")

In [ ]:
df_startup_number = pyam.IamDataFrame(
    startup_number.rename(columns={"Year.founded": "Year", "Method": "Variable", "Num.startups": "value"}),
    **iamc_args,
    region="World",
    unit="",
)

In [ ]:
df_startup_number.rename(
    variable=dict(
        {
          (
              i, "Start-Up|Removal|Number of Firms|" +
              (
                  i
                  .replace("BECCS", "Bioenergy with CCS")
                  .replace("Blue carbon", "Blue Carbon")
                  .replace("DACCS", "Direct Air Capture with CCS")
                  .replace("DOCCS", "Direct Ocean Capture")
                  .replace("Soil carbon", "Soil Carbon Sequestration")
                  .replace("Biomass burial", "Biomass Burial")
                  .replace("Enhanced weathering", "Enhanced Weathering")
                  .replace("Marine biomass", "Biomass Sinking")
                  .replace("OAE", "Ocean Alkalinity Enhancement")
                  
              )
          )
            for i in df_startup_number.variable
        },
    ),
    inplace=True,
)

In [ ]:
df_startup_number.plot.bar(stacked=True)

In [ ]:
df_startup_number.aggregate("Start-Up|Removal|Number of Firms", append=True)

In [ ]:
df_planned_projects = pyam.IamDataFrame(
    "source/3_Demonstration/JG_Chapter-3-planned-announced-projects.xlsx",
    sheet_name="Fig 3.4 - planned projects",
)

In [ ]:
df_planned_projects.rename(
    variable={
        "DACCS": "Carbon Removal|Novel|Direct Air Capture with CCS",
        "Total BECCS": "Carbon Removal|Novel|Bioenergy with CCS"
    },
    inplace=True,
)
df_planned_projects.convert_unit("t CO2", "Gt CO2/yr", factor=0.000000001, inplace=True)

In [ ]:
df_announced_projects = pyam.IamDataFrame(
    "source/3_Demonstration/JG_Chapter-3-planned-announced-projects.xlsx",
    sheet_name="Fig 3.5 - announced projects",
)

In [ ]:
df_announced_projects.rename(
    variable=dict(
        {
          (
              i, "Carbon Removal|Novel|" +
              (
                  i
                  .replace("BECCS", "Bioenergy with CCS")
                  .replace("DAC", "Direct Air Capture with CCS")
                  .replace("BECCS and DAC", "Bioenergy and Direct Air Capture with CCS")
                  .replace("sinking", "Sinking")
                  .replace("Rock ", "")
                  .replace("Ocean Alkanization", "Ocean Alkalinity Enhancement")
              )
          )
            for i in df_announced_projects.variable
        },
    ),
    inplace=True,
)

df_announced_projects.convert_unit("t CO2", "Gt CO2/yr", factor=0.000000001, inplace=True)

In [ ]:
df_announced_projects.rename(
    scenario={
        "CDR Company Announcements (Long-Term)": "Announced CDR Projects (Long-Term)"
    },
    inplace=True,
)

In [ ]:
final_df = pyam.concat(
    [df, df_startup_number, df_planned_projects, df_announced_projects]
)

In [ ]:
dsd.validate(final_df)

In [ ]:
platform = ixmp4.Platform("socdr-dev")

In [ ]:
final_df.set_meta("Chapter 3", "Chapter")

In [ ]:
for model, scenario in final_df.index:
    run = platform.runs.get(model, scenario)
    run.iamc.remove(run.iamc.tabulate())
    run.iamc.add(final_df.filter(scenario=scenario).data)

In [ ]:
#df.to_ixmp4(platform)